In [1]:
import pandas as pd 
import numpy as np
import json

import os

# Append the entire repo parent director so files therein can be accessed in notebook
import sys
import pathlib
sys.path.append(str(pathlib.Path().absolute().parent))

from src import helper
%load_ext autoreload

In [5]:
os.listdir('../data/interim/')

['City_time_series.pickle',
 'CountyCrossWalk_Zillow.pickle',
 'City_time_series.csv.pickle',
 'CountyCrossWalk_Zillow.csv.pickle']

### State_time_series & County_time_series

In [3]:
state_ts = pd.read_pickle('../data/interim/State_time_series.pickle')
county_ts = pd.read_pickle('../data/interim/County_time_series.pickle')

FileNotFoundError: [Errno 2] No such file or directory: '../data/interim/State_time_series.pickle'

In [8]:
county_ts.RegionName.unique()

array([10001, 10003,  1003, ..., 19159, 21135, 37095])

In [12]:
county_ts.sample(3)

,Date,RegionName,DaysOnZillow_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
6352,1996-08-31,36079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,119000.0,NaN,166200.0,166200.0,248400.0,NaN,NaN,NaN,NaN,NaN
430773,2015-04-30,17005,NaN,132.0,129.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276577,2010-03-31,51035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,141500.0,NaN,NaN,NaN,NaN,NaN


In [23]:
type(county_ts.loc[0, 'RegionName'])

numpy.int64

In [24]:
type(county_xwalk.loc[0, 'CountyFIPS'])

numpy.int64

#### CountyCrossWalk_Zillow
- not obvious what CBSACode is for
- FIPS is a concatenation of StateFIPS and CountyFIPS together
    - if CountyFIPS is 2 digits then there's a leading 0
    - StateFIPS is only 2 digits long

In [13]:
county_xwalk = pd.read_pickle('../data/interim/CountyCrossWalk_Zillow.pickle')
county_xwalk.shape

(3144, 10)

In [48]:
county_xwalk.head(3)

,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,FIPS,CBSACode
0,Pike,Pennsylvania,42,103,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",280,394913.0,42103,35620.0
1,Bronx,New York,36,5,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",401,394913.0,36005,35620.0
2,Essex,New Jersey,34,13,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",504,394913.0,34013,35620.0


**Figuring out which column from county_xwalk maps over to county_ts**
- FIPS is the common column b/t the 2 dataframes that we can join on
- FIPS will allow us to access the County name, State, etc. in the `county_xwalk` DataFrame which does not appear in the `county_ts` DataFrame

In [34]:
xwalk_countyFIPS = county_xwalk['CountyFIPS'].unique().tolist()
xwalk_countyRegionID = county_xwalk['CountyRegionID_Zillow'].unique().tolist()
xwalk_Metro = county_xwalk['MetroRegionID_Zillow'].unique().tolist()
xwalk_FIPS = county_xwalk['FIPS'].unique().tolist()
xwalk_CBSACode = county_xwalk['CBSACode'].unique().tolist()

print(county_ts['RegionName'].isin(xwalk_countyFIPS).sum())
print(county_ts['RegionName'].isin(xwalk_countyRegionID).sum())
print(county_ts['RegionName'].isin(xwalk_Metro).sum())
print(county_ts['RegionName'].isin(xwalk_FIPS).sum())
print(county_ts['RegionName'].isin(xwalk_CBSACode).sum())

0
9603
0
518791
0


In [36]:
# Create a dictionary with {key=FIP: value=CountyName}
countyname_by_FIPS = dict(zip(county_xwalk['FIPS'], county_xwalk['CountyName']))

In [37]:
county_ts.head(3)

,Date,RegionName,DaysOnZillow_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
0,1996-04-30,10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65600.0,NaN,85700.0,86000.0,129700.0,NaN,NaN,NaN,NaN,NaN
1,1996-04-30,10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81900.0,NaN,116800.0,116900.0,181400.0,NaN,NaN,NaN,NaN,NaN
2,1996-04-30,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70600.0,NaN,110600.0,106800.0,192300.0,NaN,NaN,NaN,NaN,NaN


In [40]:
# Use the `countyname_by_FIPS` dictionary to impute a new column in the `county_ts` DataFrame

countyName_column = []
for row in county_ts.itertuples():
    if row.RegionName in countyname_by_FIPS:
        countyName_column.append(countyname_by_FIPS[row.RegionName])
    else:
        countyName_column.append(np.nan)
        
assert len(countyName_column) == len(county_ts), 'mismatched lengths'

county_ts['CountyName'] = countyName_column
county_ts[['RegionName', 'CountyName']].head(3)

,RegionName,CountyName
0,10001,Kent
1,10003,New Castle
2,1003,Baldwin


In [43]:
county_ts[['RegionName', 'CountyName']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518791 entries, 0 to 518790
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   RegionName  518791 non-null  int64 
 1   CountyName  518791 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.9+ MB


In [47]:
county_ts['RegionName'].nunique()

2701

In [46]:
county_ts['CountyName'].nunique()

1614